In [1]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF, BPR, SANSA, BiVAECF, LightGCN, RecVAE, EASE, NGCF, VAECF, IBPR, NeuMF, HPF
from cornac.metrics import Precision, Recall, NDCG, MAP

/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Import datasets

In [2]:
ml_100k = cornac.datasets.movielens.load_feedback()
rs = RatioSplit(data=ml_100k, test_size=0.2, rating_threshold=4.0, seed=123)

### 2. Initialize models, here we are comparing: Biased MF, PMF, and BPR

In [3]:
mf = MF(k=10, max_iter=200, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)
pmf = PMF(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.001, seed=123)
bpr = BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)
sansa_cholmod = SANSA(
    name="SANSA (CHOLMOD)",
    l2=500.0,
    weight_matrix_density=1e-2,
    compute_gramian=True,
    factorizer_class="CHOLMOD",
    factorizer_shift_step=1e-3,
    factorizer_shift_multiplier=2.0,
    inverter_scans=5,
    inverter_finetune_steps=20,
    use_absolute_value_scores=False,
)

sansa_icf = SANSA(
    name="SANSA (ICF)",
    l2=10.0,
    weight_matrix_density=1e-2,
    compute_gramian=True,
    factorizer_class="ICF",
    factorizer_shift_step=1e-3,
    factorizer_shift_multiplier=2.0,
    inverter_scans=5,
    inverter_finetune_steps=20,
    use_absolute_value_scores=False,
)

bivaecf = BiVAECF(
    k=10,
    use_gpu=False,
)

lightgcn = LightGCN()

recvae = RecVAE(
    use_gpu=False,
)

ease = EASE()

ngcf = NGCF()

vaecf = VAECF(
    use_gpu=False,
)

ibpr = IBPR()

neumf = NeuMF()

hpf = HPF()

models = [mf, pmf, bpr, bivaecf, recvae, ease, vaecf, ibpr, neumf, hpf]#sansa_cholmod, sansa_icf, lightgcn, ngcf]

### 3. Evaluation metrics

In [4]:
metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP()]

In [5]:
cornac.Experiment(
    eval_method=rs,
    models=models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

100%|██████████| 20/20 [03:10<00:00,  9.53s/it, loss=0.247]


Learning...
Learning completed!

TEST:
...
        |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------- + ------ + ------- + ------------ + --------- + --------- + --------
MF      | 0.0290 |  0.0214 |       0.0218 |    0.0165 |    0.1320 |   0.2939
PMF     | 0.0538 |  0.0744 |       0.0618 |    0.0493 |    2.1057 |   0.2786
BPR     | 0.1022 |  0.1435 |       0.1077 |    0.1214 |    1.0631 |   0.2825
BiVAECF | 0.1471 |  0.1952 |       0.1483 |    0.1633 |    7.7278 |   0.2894
RecVae  | 0.1313 |  0.1697 |       0.1294 |    0.1489 |   38.0527 |   0.6233
EASEᴿ   | 0.1743 |  0.2327 |       0.1630 |    0.2078 |    0.1140 |   0.3299
VAECF   | 0.1497 |  0.1986 |       0.1512 |    0.1650 |    3.5425 |   0.3678
IBPR    | 0.0981 |  0.1143 |       0.0931 |    0.0889 |  160.5502 |   0.2780
NeuMF   | 0.1649 |  0.2130 |       0.1570 |    0.1839 |  194.7774 |   3.9372
HPF     | 0.1730 |  0.2386 |       0.1782 |    0.1871 |    4.4713 |   0.2992



In [6]:
import torch
torch.__version__

'2.7.0'